In [1]:
import copy
import glob
import gzip
import json
import os
import shutil
import sys
import time
import warnings
import zipfile
from datetime import date

import cdsapi
import h5py
import numpy
import numpy as np
import pandas as pd
import urllib3
import xarray as xr

sys.path.append(os.getcwd() + "/../cds-backend/code/")
import cds_eua4 as eua

warnings.filterwarnings("ignore")

sys.path.insert(0, os.getcwd() + "/../resort/rasotools-master/")
import matplotlib
import matplotlib.font_manager as font_manager
import matplotlib.pylab as plt
import matplotlib.pyplot as maplt
import rasotools
import seaborn

matplotlib.rcParams.update({"font.size": 20})
matplotlib.rcParams["figure.figsize"] = (20, 10)
font = {
    "size": 20,
}
matplotlib.rc("font", **font)

import trajectory as trj

no config found


In [2]:
import cdsapi

c = cdsapi.Client()

r = c.retrieve(
    'insitu-observations-igra-baseline-network',
    {
        'archive_type': 'harmonized_global_radiosonde_archive',
        'format': 'csv-lev.zip',
        'year': '2000',
        'month': '01',
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'area': [
            60, 10, 40,
            20,
        ],
        'variable': [
            'air_temperature', 'eastward_wind_component', 'northward_wind_component',
            'relative_humidity',
        ],
    },
    'download.csv-lev.zip')

2023-07-19 16:48:53,497 INFO Welcome to the CDS
2023-07-19 16:48:53,498 INFO Sending request to https://cds-test.copernicus-climate.eu/api/v2/resources/insitu-observations-igra-baseline-network
2023-07-19 16:48:53,604 INFO Request is completed
2023-07-19 16:48:53,605 INFO Downloading https://cds-test-download-0000.copernicus-climate.eu/cache-compute-0000/cache/data1/adaptor.insitu_reference.retrieve-1689759763.2069223-31328-1-d402cc8a-aac5-40d6-a0ba-2817bded26f5.zip to download.csv-lev.zip (548.2K)
2023-07-19 16:48:54,086 INFO Download rate 1.1M/s                     


In [3]:
tempdir = "./tmp/"
delete_dl = True
with zipfile.ZipFile("download.csv-lev.zip", "r") as zip_ref:
    try:
        os.mkdir(tempdir)
    except:
        pass

    zip_ref.extractall(tempdir)

for i in glob.glob(tempdir + "*"):
    print(i)
    df = pd.read_csv(i, header=16)

# if delete_dl:
#     shutil.rmtree(tempdir)

./tmp/IGRA_H_20000101_20000131_subset_cdm-lev_v1.csv


In [4]:
df

,station_name,radiosonde_code,sensor_model,report_timestamp,actual_time,report_id,longitude,latitude,height_of_station_above_sea_level,air_pressure,air_temperature,relative_humidity,eastward_wind_component,northward_wind_component
0,AUM00011240,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,2000-01-01 03:44:00+00,82731,15.4392,46.9931,340.0,10000,215.65,NaN,6.06,-3.50
1,AUM00011240,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,2000-01-01 03:44:00+00,82731,15.4392,46.9931,340.0,15000,216.85,NaN,16.74,2.95
2,AUM00011240,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,2000-01-01 03:44:00+00,82731,15.4392,46.9931,340.0,20000,216.05,NaN,28.98,7.76
3,AUM00011240,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,2000-01-01 03:44:00+00,82731,15.4392,46.9931,340.0,22800,216.45,1.93,45.11,16.42
4,AUM00011240,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,2000-01-01 03:44:00+00,82731,15.4392,46.9931,340.0,25000,219.05,2.14,51.68,18.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44295,GMM00010184,62.0,Vaisala RS80/PCCora (Finland),2000-01-31 12:00:00+00,2000-01-31 10:52:00+00,11949459,13.4075,54.0978,2.0,92500,275.66,80.40,7.14,-27.09
44296,GMM00010184,62.0,Vaisala RS80/PCCora (Finland),2000-01-31 12:00:00+00,2000-01-31 10:52:00+00,11949459,13.4075,54.0978,2.0,97400,279.56,62.01,NaN,NaN
44297,GMM00010184,62.0,Vaisala RS80/PCCora (Finland),2000-01-31 12:00:00+00,2000-01-31 10:52:00+00,11949459,13.4075,54.0978,2.0,100000,281.50,65.21,3.48,-17.32
44298,GMM00010771,62.0,Vaisala RS80/PCCora (Finland),2000-01-09 00:00:00+00,2000-01-08 22:47:00+00,5848365,11.9022,49.4283,417.0,96600,272.85,95.02,NaN,NaN


In [5]:
lat_disp, lon_disp, sec_disp = np.array([np.nan]*len(df)),np.array([np.nan]*len(df)),np.array([np.nan]*len(df))
for station in df.station_name.drop_duplicates():
    df_i = df[df.station_name == station]
    for rid in df_i.report_id.drop_duplicates():
        df_j = df_i[df_i.report_id == rid]
        df_j_cleanded = df_j.sort_values(by='air_pressure', ascending=False).dropna(subset=['air_temperature', 'eastward_wind_component', 'northward_wind_component'])
        if len(df_j_cleanded) > 3:
            
            idx =  df_j_cleanded.index.values
            lat_i, lon_i, sec_i = trj.trajectory(df_j_cleanded.latitude.iloc[0], 
                                                 df_j_cleanded.longitude.iloc[0], 
                                                 df_j_cleanded.eastward_wind_component.values, 
                                                 df_j_cleanded.northward_wind_component.values, 
                                                 df_j_cleanded.air_pressure.values, 
                                                 df_j_cleanded.air_temperature.values
                                                )
            lat_disp[idx] = lat_i
            lon_disp[idx] = lon_i
            sec_disp[idx] = sec_i
    
df['latitude_displacement'] = lat_disp
df['longitude_displacement'] = lon_disp
df['time_displacement'] = sec_disp

In [7]:
df

,station_name,radiosonde_code,sensor_model,report_timestamp,actual_time,report_id,longitude,latitude,height_of_station_above_sea_level,air_pressure,air_temperature,relative_humidity,eastward_wind_component,northward_wind_component,latitude_displacement,longitude_displacement,time_displacement
0,AUM00011240,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,2000-01-01 03:44:00+00,82731,15.4392,46.9931,340.0,10000,215.65,NaN,6.06,-3.50,0.172333,0.895818,3116.974654
1,AUM00011240,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,2000-01-01 03:44:00+00,82731,15.4392,46.9931,340.0,15000,216.85,NaN,16.74,2.95,0.173629,0.818638,2603.666017
2,AUM00011240,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,2000-01-01 03:44:00+00,82731,15.4392,46.9931,340.0,20000,216.05,NaN,28.98,7.76,0.156123,0.708764,2239.128908
3,AUM00011240,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,2000-01-01 03:44:00+00,82731,15.4392,46.9931,340.0,22800,216.45,1.93,45.11,16.42,0.138112,0.627770,2073.249551
4,AUM00011240,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,2000-01-01 03:44:00+00,82731,15.4392,46.9931,340.0,25000,219.05,2.14,51.68,18.81,0.119531,0.552894,1955.825774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44295,GMM00010184,62.0,Vaisala RS80/PCCora (Finland),2000-01-31 12:00:00+00,2000-01-31 10:52:00+00,11949459,13.4075,54.0978,2.0,92500,275.66,80.40,7.14,-27.09,-0.025364,0.010319,127.141079
44296,GMM00010184,62.0,Vaisala RS80/PCCora (Finland),2000-01-31 12:00:00+00,2000-01-31 10:52:00+00,11949459,13.4075,54.0978,2.0,97400,279.56,62.01,NaN,NaN,NaN,NaN,NaN
44297,GMM00010184,62.0,Vaisala RS80/PCCora (Finland),2000-01-31 12:00:00+00,2000-01-31 10:52:00+00,11949459,13.4075,54.0978,2.0,100000,281.50,65.21,3.48,-17.32,0.000000,0.000000,0.000000
44298,GMM00010771,62.0,Vaisala RS80/PCCora (Finland),2000-01-09 00:00:00+00,2000-01-08 22:47:00+00,5848365,11.9022,49.4283,417.0,96600,272.85,95.02,NaN,NaN,NaN,NaN,NaN


In [6]:
lat_disp, lon_disp, sec_disp = [], [], []
for station in df.station_name.drop_duplicates():
    df_i = df[df.station_name == station]
    for rid in df_i.report_id.drop_duplicates():
        df_j = df_i[df_i.report_id == rid]
        lat_disp_j, lon_disp_j, sec_disp_j = np.array([np.nan]*len(df_j)),np.array([np.nan]*len(df_j)),np.array([np.nan]*len(df_j))
        df_j_cleanded = df_j.sort_values(by='air_pressure', ascending=False).dropna(subset=['air_temperature', 'eastward_wind_component', 'northward_wind_component'])
        if len(df_j_cleanded) > 3:

            df_j_cleanded = df_j_cleanded.reset_index()
            idx =  df_j_cleanded.index.values
            lat_i, lon_i, sec_i = trj.trajectory(df_j_cleanded.latitude.iloc[0], 
                                                 df_j_cleanded.longitude.iloc[0], 
                                                 df_j_cleanded.eastward_wind_component.values, 
                                                 df_j_cleanded.northward_wind_component.values, 
                                                 df_j_cleanded.air_pressure.values, 
                                                 df_j_cleanded.air_temperature.values
                                                )
            lat_disp_j[idx] = lat_i
            lon_disp_j[idx] = lon_i
            sec_disp_j[idx] = sec_i
            
        df_j['latitude_displacement'] = lat_disp_j
        df_j['longitude_displacement'] = lon_disp_j
        df_j['time_displacement'] = sec_disp_j
        lat_disp.append(df_j['latitude_displacement'].interpolate().values)
        lon_disp.append(df_j['longitude_displacement'].interpolate().values)
        sec_disp.append(df_j['time_displacement'].interpolate().values)
    
df['latitude_displacement_interpolated'] = lat_disp
df['longitude_displacement_interpolated'] = lon_disp
df['time_displacement_interpolated'] = sec_disp

ValueError: Length of values (1017) does not match length of index (44300)

In [8]:
df_j

,station_name,radiosonde_code,sensor_model,report_timestamp,actual_time,report_id,longitude,latitude,height_of_station_above_sea_level,air_pressure,air_temperature,relative_humidity,eastward_wind_component,northward_wind_component,latitude_displacement,longitude_displacement,time_displacement
42349,ITM00016144,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-30 12:00:00+00,2000-01-30 11:00:00+00,3992829,11.6225,44.6539,10.0,3000,212.48,NaN,2.47,-25.33,0.000000,0.000000,0.000000
42350,ITM00016144,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-30 12:00:00+00,2000-01-30 11:00:00+00,3992829,11.6225,44.6539,10.0,3410,207.80,1.03,NaN,NaN,-0.009269,-0.000113,127.628834
42351,ITM00016144,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-30 12:00:00+00,2000-01-30 11:00:00+00,3992829,11.6225,44.6539,10.0,4300,205.24,1.09,9.94,-16.98,-0.026856,0.008169,267.276269
42352,ITM00016144,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-30 12:00:00+00,2000-01-30 11:00:00+00,3992829,11.6225,44.6539,10.0,5000,206.31,NaN,13.73,-9.28,-0.065481,0.070896,581.924688
42353,ITM00016144,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-30 12:00:00+00,2000-01-30 11:00:00+00,3992829,11.6225,44.6539,10.0,5570,205.55,0.92,NaN,NaN,-0.123165,0.246921,1101.844631
42354,ITM00016144,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-30 12:00:00+00,2000-01-30 11:00:00+00,3992829,11.6225,44.6539,10.0,6630,211.81,0.00,NaN,NaN,-0.154281,0.396431,1426.861097
42355,ITM00016144,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-30 12:00:00+00,2000-01-30 11:00:00+00,3992829,11.6225,44.6539,10.0,7000,210.63,NaN,8.10,-15.13,-0.199743,0.589135,1821.044303
42356,ITM00016144,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-30 12:00:00+00,2000-01-30 11:00:00+00,3992829,11.6225,44.6539,10.0,7800,206.63,0.98,NaN,NaN,-0.238452,0.704867,2057.414377
42357,ITM00016144,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-30 12:00:00+00,2000-01-30 11:00:00+00,3992829,11.6225,44.6539,10.0,9740,215.12,0.00,NaN,NaN,-0.277559,0.852212,2333.796527
42358,ITM00016144,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-30 12:00:00+00,2000-01-30 11:00:00+00,3992829,11.6225,44.6539,10.0,10000,213.45,NaN,19.20,-11.78,-0.280143,0.865444,2358.209581


In [ ]:
df[:10]